__Fetch relevant files from stacks populations run__

In [ ]:
%%bash

ln -s /media/chrishah/STORAGE/RAD/stacks/ALL/mapping/excl_PCR_dupl/BWA-8MM/M_zebra/3-populations/Diplotaxodon_4pop/m5_mpop5_kernel_iterate_ONE_SNP_PER_TAG/r_0.8-p_4/batch_1.vcf.gz .
ln -s /media/chrishah/STORAGE/RAD/stacks/ALL/mapping/excl_PCR_dupl/BWA-8MM/M_zebra/3-populations/Diplotaxodon_4pop/populationmap .
mkdir matrix

__create 10 Bayenv input files with 5000 randomly selected loci in each__

In [ ]:
%%bash
#pip install pyvcf

for a in {1..10}
do
    echo -e "\nrepitition $a:\n"
    python /home/chrishah/Dropbox/Github/genomisc/popogeno/vcf_2_bayenv.py batch_1.vcf.gz --min_number 6 -r 5000 -o matrix/random_5000_rep_$a -m populationmap
done

__create 10 covariance matrizes with 100000 iterations each__

In [ ]:
%%bash
cd matrix/

for a in {1..10}
do
    /home/chrishah/src/Bayenv/bayenv2 0 -p 4 -r -$RANDOM -k 100000 -i random_5000_rep_$a.bayenv.SNPfile > random_5000_rep_$a.log
done

__construct average covariance matrix from 10 random sets__

In [ ]:
import numpy as np
main_list = []

for a in range(10):
    current = "matrix/random_5000_rep_"+str(a+1)+"_it-1e5.matrix"
#    print current
    IN = open(current,"r")
    temp_list = []
    for line in IN:
        temp_list.extend(line.rstrip().split("\t"))

    for i in range(len(temp_list)):
        if a == 0:
            main_list.append([float(temp_list[i])])
        else:
            main_list[i].append(float(temp_list[i]))
        
#print main_list

av_out_list = []
std_out_list = []
for j in range(len(main_list)):
    av_out_list.append(np.mean(main_list[j]))

#print av_out_list

outstring = ""
for z in range(len(av_out_list)):
    av_out_list[z] = "%s\t" %av_out_list[z]
    if not outstring:
        outstring = av_out_list[z]
    else:
        outstring = outstring+av_out_list[z]
        if ((z+1) % 4 == 0):
            outstring = "%s\n" %(outstring)


OUT = open("matrix/av_matrix.matrix","w")
OUT.write(outstring)
OUT.close()
        

__Prepare__ environmental data - average and normalize

raw data is provided in a csv file with the first column containing the population id. See example in test-data.

In [ ]:
def normalize (csv, norm_prefix="", normalize=True, boxplot=False, boxplots_prefix=""):
    """
    The function parses a csv file and outputs normalized values
    and boxplots if desired
    """
    from collections import defaultdict
    import numpy as np
    import subprocess
    
    if normalize and not norm_prefix:
        raise IOError("You have to specify a prefix for the output files containing the normalized data - use 'norm_prefix='")
        
    if boxplot and not boxplots_prefix:
        raise IOError("You have to specify a prefix for the boxplot files to be produced - use 'boxplots_prefix='")
            
    populations = []    
    columns = {}
    indices = defaultdict(list)
    normalized = defaultdict(list)
    
    IDs = []
    pops = []
    
    INFILE = open(csv, 'r')

    headers = INFILE.readline().strip().split(",")
#    print headers
    headers.pop(0)
    IDs = sorted(headers)
    for env in headers:
        columns[env] = []
        
    for line in INFILE:
        line = line.strip()
        temp=line.split(",")
        populations.append(temp.pop(0))
#        print population
        for i in range(len(temp)):
#            print "%i:%s\n" %(i, header[i])
            columns[headers[i]].append(float(temp[i]))
#        print columns
 
    pops = list(sorted(set(populations)))
#    print pops
    #find indexes for each
    for pop in pops:
#        print "finding indices for %s" %pop
        for i in range(len(populations)):
            if pop == populations[i]:
#                print i
                indices[pop].append(i)

#    print indices
    
#    print "\nCalculating means\n"
    for env in headers:
        per_pop = {}
        for pop in pops:
#            print pop
#            print "%s - should be %i" %(env, len(indices[pop]))       
            per_pop_list = []
            for i in indices[pop]:
                per_pop_list.append(columns[env][i])
                
#            print per_pop_list
#            print "%s mean: %s" %(pop, np.mean(per_pop_list))
#            print "%s mean: %s" %(env, np.mean(columns[env]))
#            print "%s sd: %s" %(env, np.std(columns[env]))
            per_pop[pop] = per_pop_list
            norm = (np.mean(per_pop_list) - np.mean(columns[env])) / np.std(columns[env])
#            print norm
            normalized[env].append(norm)
        if boxplot:
            print "Creating boxplot for %s\n" %env
            Rscript = boxplots_prefix+env+'.R'
            FH = open(Rscript, 'w')
            for pop in pops:
#                print per_pop[pop]
                vector = ""
                for v in per_pop[pop]:
                    vector+=repr(v)+','
                FH.write(pop+' <- c(%s)\n' %vector[:-1])
            FH.write("svg(filename = '"+boxplots_prefix+env+".svg')\n") #svg(filename = '$prefix-top-$cutoff.svg')
            FH.write("boxplot(%s, names = c('%s'), main = '%s')\n" %(", ".join(pops), "', '".join(pops), env)) 
            FH.write("dev.off()\n")
            FH.close()
            c = subprocess.Popen(['Rscript', Rscript], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            (output,err) = c.communicate()

            if err:
                print err
#            else:
#                print output
            
#    print normalized
    if normalize:
        print "\nnormalizing %s environmental factors across %s populations\nwriting to:\n\t%s.bayenv\n\t%s.csv" %(len(IDs), len(pops), norm_prefix, norm_prefix)
        OUTCSV = open(norm_prefix+'.csv', 'w')
        OUTCSV.write(",%s\n" %",".join(pops))
        OUTBAYENV = open(norm_prefix+'.bayenv',"w")
        for env in sorted(columns.keys()):
            outstring = ""
#            IDs.append(env)
            for n in normalized[env]:
                outstring += repr(n)+"\t"

    #        print outstring
            OUTBAYENV.write(outstring+"\n")
            OUTCSV.write("%s,%s\n" %(env, outstring.replace('\t',',')[:-1]))
        
        OUTBAYENV.close()
        OUTCSV.close()
    
    return pops, IDs
        


In [ ]:
populations, IDs = normalize(csv='../Diplotaxodon_Morphometric_Data_raw.csv', normalize=True, norm_prefix='Diplotaxodon_Morphometric_Data_normalized', boxplot=False)

In [ ]:
print populations
print IDs

__convert vcf to bayenv - generate full SNP files__

In [ ]:
%%bash

mkdir SNPfiles
python /home/chrishah/Dropbox/Github/genomisc/popogeno/vcf_2_div.py ../batch_1.vcf.gz --min_number 6 -o SNPfiles/full_set -m ../populationmap

__split up SNPfiles into single files__

In [ ]:
def split_for_Bayenv(infile, out_prefix):
    """
    This function takes a bayenv formatted multi-SNP file,
    splits it up into separate files (one SNP per file).
    """
    SNPcount = 0
    temp = []
    IN = open(infile, 'r')
    for line in IN:
        line = line.strip()
        temp.append(line)
        SNPcount+=1
        if (SNPcount % 2 == 0):
            OUT = open('%s-%07d.txt' %(out_prefix, SNPcount/2), 'w') #out_prefix+'-'+str(SNPcount/2)+'.txt', 'w')
#            print "%s\t\n" %"\n".join(temp)
            OUT.write("%s\t\n" %"\t\n".join(temp))
            OUT.close()
            temp = []
            
            

In [ ]:
split_for_Bayenv(infile='SNPfiles/full_set.bayenv.SNPfile', out_prefix='SNPfiles/Diplo_SNP')

__Run Bayenv for 10 replications__ serially

In [ ]:
#find the number of SNP files to add to specify in loop below
!ls -1 SNPfiles/SNP-* |wc -l

In [ ]:
!mkdir running_Bayenv

In [ ]:
%%bash
#adjust bayenv command to your requirements

iterations=1000000

cd running_Bayenv/

for rep in {1..10}; do ran=$RANDOM; for a in {0000001..0021968}; do /home/chrishah/src/Bayenv/bayenv2 -i ../SNPfiles/SNP-$a.txt -e ../Nyassochromis_normalized.bayenv -m ../matrix/av_matrix.matrix -k $iterations -r -$ran -p 3 -n 14 -t -X -o bayenv_out_k100000_env_rep_$rep-rand_$ran; done > log_rep_$rep; done

__ALTERNATIVE__

__Bayenv__ can be run on a HPC cluster in parallel. I provide a script `submit_Bayenv_array_multi.sh` that I used to run 10 replicates as arrayjob on a cluster that was running a PBS scheduling system. Total runtime for 10 replicates with 1M Bayenv iterations/SNP was ~ 24h. The results from the individual runs were then concatenated with the script `concat_sorted.sh` and moved to the directory `running_Bayenv` on the local machine.

#ANALYSE RANK STATISTICS#
please make sure you load all functions below first

__Calculating RANK STATISTICS__

In [ ]:
mkdir RANK_STATISTIC/

In [ ]:
#create the list of Bayenv results files to be processed
import os

bayenv_res_dir = './running_bayenv/'
bayenv_files = []

for fil in os.listdir(bayenv_res_dir):
    if fil.endswith(".bf"):
        print(bayenv_res_dir+"/"+fil)
        bayenv_files.append(bayenv_res_dir+"/"+fil)

In [ ]:
print bayenv_files
print "\n%i" %len(bayenv_files)
print IDs

In [ ]:

rank_results = calculate_rank_stats(SNP_map="SNPfiles/full_set.bayenv.SNPmap", infiles = bayenv_files, ids = IDs, prefix = 'RANK_STATISTIC/Diplo_k_1M')

__CREATE POPE PLOTS and extract the SNP ids in the top 5 percent__

In [ ]:
print IDs

In [ ]:
full_rank_files = []

file_dir = 'RANK_STATISTIC/'
for id in IDs:
#    print id
    for file in os.listdir(file_dir):
        if file.endswith('_'+id+'.txt'):
#            print [id,file_dir+'/'+file]
            full_rank_files.append([id,file_dir+'/'+file])
            break
        

In [ ]:
print full_rank_files

In [ ]:
plot_pope(files_list=full_rank_files, cutoff=0.95, num_replicates=10)

__CREATE POPE PLOTS and extract the SNP ids in the top 1 percent__

In [ ]:
plot_pope(files_list=full_rank_files, cutoff=0.99, num_replicates=10)

__find genes__ up and downstream of correlated SNPs

In [ ]:
#make list desired rank statistic tsv files

import os

file_dir = 'RANK_STATISTIC/'

rank_stats_files = []

for file in os.listdir(file_dir):
    if file.endswith('.tsv'):
            print file_dir+'/'+file
            rank_stats_files.append(file_dir+'/'+file)

In [ ]:
gff_per_scaffold = parse_gff(gff='../../../../Nyassochromis/3_populations/Metriaclima_zebra.BROADMZ2.gtf')

In [ ]:
genes_per_analysis = find_genes(rank_stats = rank_stats_files, gff = gff_per_scaffold, distance = 15)

In [ ]:
annotate_genes(SNPs_to_genes=genes_per_analysis, annotations='../../../../Nyassochromis/3_populations/blast2go_table_20150630_0957.txt')

In [ ]:
mkdir find_genes

In [ ]:
write_candidates(SNPs_to_genes=genes_per_analysis, whitelist=genes_per_analysis.keys(), out_dir='./find_genes/')

__remove the most extreme Bayenv results and recalculate rank stats__

In [ ]:
mkdir RANK_STATISTIC_reduced

In [ ]:
exclude_extreme_rep(dictionary = rank_results, ids = IDs, prefix = 'RANK_STATISTIC_reduced/Diplotaxodon_reduced')

In [ ]:
reduced_rank_files = []

file_dir = 'RANK_STATISTIC_reduced/'
for id in IDs:
#    print id
    for file in os.listdir(file_dir):
        if '_'+id+'_ex_rep' in file and file.endswith('.txt'):
#            print [id,file_dir+'/'+file]
            reduced_rank_files.append([id,file_dir+'/'+file])
            break
        

In [ ]:
print reduced_rank_files

In [ ]:
plot_pope(files_list=reduced_rank_files, cutoff=0.95, num_replicates=9)

__find genes__ up and downstream of correlated SNPs

In [ ]:
#make list desired rank statistic tsv files

import os

file_dir = 'RANK_STATISTIC_reduced/'

rank_stats_files = []

for file in os.listdir(file_dir):
    if file.endswith('.tsv'):
            print file_dir+'/'+file
            rank_stats_files.append(file_dir+'/'+file)

In [ ]:
genes_per_analysis = find_genes(rank_stats = rank_stats_files, gff = gff_per_scaffold, distance = 15)

In [ ]:
annotate_genes(SNPs_to_genes=genes_per_analysis, annotations='../../../../Nyassochromis/3_populations/blast2go_table_20150630_0957.txt')

In [ ]:
mkdir find_genes_reduced/

In [ ]:
write_candidates(SNPs_to_genes=genes_per_analysis, whitelist=genes_per_analysis.keys(), out_dir='./find_genes_reduced/')

## FUNCTIONS ##

###DEFINE SOME FUNCTIONS FOR CALCULATING RANK STATS###

__Define the function__ to calculate averages and standard deviations of bayesfactor ranks across replicates. 

In [ ]:
def calculate_rank_stats(SNP_map, infiles, ids, prefix): #these options are currently not implemented, threshold = 0.01, window = 50e3, sigma_factor = 3, bootstrap_rep = 100):
    """
    The function will calculate rank statistics (averages, standard deviations, etc)
    across a number of Bayenv replicates
    """
    #import 
    from collections import defaultdict
    import os
    import numpy as np
    
    #define global variables
    global_dict = defaultdict(dict)
    glob = defaultdict(dict) #holds all information for the SNPs
    reps = []
    extremes = {}
    rep_count = 0
    return_dict = {}
    
    #read in SNP_map
    SNPmap = open(SNP_map, 'r')
    i=0
    for line in SNPmap:
        global_dict[i]['chrom'] = line.strip()
        i += 1
    print "Total number of SNPs (according to the SNPmap): %i" %len(global_dict)
    
    #assess Bayenv files
    print "Number of Bayenv replicates: %i" %len(infiles)
    
    #assess environmental factors
    print "Number of environmental factors analysed: %i" %len(ids)

    #display settings
#    print "Sliding window settings:"
#    print "\tWindow size: %i bp" %window
#    print "\tSigma: %s" %sigma_factor

    #start processing
    print "parsing bayenv files"
    for bayenv_file in sorted(infiles):
        
        print "\nprocessing replicate %i:\n%s" %(rep_count, bayenv_file)
        reps.append(bayenv_file)
        fh = open(bayenv_file,'r')
        j=0 #this variable will hold the index of the current SNP
        sorting = defaultdict(dict) #This dictionary will contain the rank sorted SNPs
        for bf in fh:
            for factor_index in range(len(ids)):
#                print j
                factor = ids[factor_index]
#                print "processing factor %s" %factor
                if not glob.has_key(factor):
                    glob[factor] = defaultdict(dict)
            
                if not sorting[factor].has_key(float(bf.split('\t')[1+factor_index])):
                    sorting[factor][float(bf.split('\t')[1+factor_index])]=[j]
                else:
                    sorting[factor][float(bf.split('\t')[1+factor_index])].append(j)
            j += 1

        
        fh.close()
        
        for factor in sorting.keys(): #do sorting and add rank information to global dictionary
#            print factor
            rank = 0
            for r in sorted(sorting[factor].keys()):
##                print "rank: %i" %rank
##                print "bf: %f" %r
##                print sorting[r]
##                print len(sorting[r])
                for SNP in sorting[factor][r]:
##                    print SNP
                    if not glob[factor].has_key(SNP):
                        glob[factor][SNP]['ranks'] = [rank]
                    else:
                        glob[factor][SNP]['ranks'].append(rank)
                rank += len(sorting[factor][r])
        rep_count += 1
            
            
    output_columns = ['avg_rank', 'med_rank', 'std_rank', 'var_rank', 'mad_rank', 'avg_rank_rel', 'med_rank_rel', 'var_rank_rel', 'var_rank_weight', 'var_weighted_avg_rank', 'var_weighted_rel_avg_rank']
                
    print "\nSUMMARY:\n"
    for fac in sorted(glob.keys()):
#        print "%s: %i" %(fac, len(glob[fac]))
        extremes[fac] = defaultdict(int)
        variances = []
#        for SNPid in glob[fac].keys()[0:10]:
#            print "%s: %s" %(SNPid, glob[fac][SNPid]['ranks'])
        for SNPid in glob[fac].keys():
#            print glob[fac][SNPid]['ranks']
#            print absolute_deviation_from_median(data=glob[fac][SNPid]['ranks'])
#            print find_max(absolute_deviation_from_median(data=glob[fac][SNPid]['ranks']))
            for maxi in find_max(absolute_deviation_from_median(data=glob[fac][SNPid]['ranks'])):
                extremes[fac][maxi] += 1
#            print extremes
            glob[fac][SNPid]['avg_rank'] = np.mean(glob[fac][SNPid]['ranks'])
            glob[fac][SNPid]['med_rank'] = np.median(glob[fac][SNPid]['ranks'])
            glob[fac][SNPid]['std_rank'] = np.std(glob[fac][SNPid]['ranks'])
            glob[fac][SNPid]['var_rank'] = np.var(glob[fac][SNPid]['ranks'])
            glob[fac][SNPid]['mad_rank'] = mad(data=glob[fac][SNPid]['ranks'])
            variances.append(glob[fac][SNPid]['var_rank'])
            glob[fac][SNPid]['avg_rank_rel'] = np.mean(glob[fac][SNPid]['ranks'])/len(glob[fac])
            glob[fac][SNPid]['med_rank_rel'] = np.median(glob[fac][SNPid]['ranks'])/len(glob[fac])
        #find maximum variance for the current factor
        max_var = max(variances)

        print "Wrting stats to %s" %(prefix+'_'+fac+'.txt')
        OUT = open(prefix+'_'+fac+'.txt','w')
        OUT.write("chrom\tbp\tSNPID\t"+"\t".join(output_columns)+'\n')

        for SNPid in glob[fac].keys(): #calculate relative and weighted variances
            glob[fac][SNPid]['var_rank_rel'] = glob[fac][SNPid]['var_rank'] / max_var
            glob[fac][SNPid]['var_rank_weight'] = 1-glob[fac][SNPid]['var_rank_rel']
            glob[fac][SNPid]['var_weighted_avg_rank'] = glob[fac][SNPid]['var_rank_weight'] * glob[fac][SNPid]['avg_rank']
            glob[fac][SNPid]['var_weighted_rel_avg_rank'] = glob[fac][SNPid]['var_rank_weight'] * glob[fac][SNPid]['avg_rank_rel']

            temp_list = []
#            outstring = str(SNPid)+','
            outstring = global_dict[SNPid]['chrom']+'\t'
            for column in output_columns:
#                print column
#                print glob[fac][SNPid][column]
                temp_list.append(str(glob[fac][SNPid][column]))
            outstring += "\t".join(temp_list)
            OUT.write(outstring+'\n')

        OUT.close()
        

        counts = [extremes[fac][i] for i in sorted(extremes[fac])]
        perc = [float(counts[i])/len(glob[fac])*100 for i in range(len(counts))]
        ex = find_max([extremes[fac][i] for i in sorted(extremes[fac])])
        print "factor %s\treplicate %s gave the most extreme ranks for %.2f %% of the SNPs" %(fac, ex[0], perc[ex[0]])

    return_dict['global'] = glob
    return_dict['extremes'] = extremes
    return_dict['SNPids'] = global_dict
    return return_dict

In [ ]:
def mad(data):
    """
    find the 'median absolute deviation'
    """
    import numpy as np
   
    return np.median(np.abs(data - np.median(data)))

In [ ]:
def absolute_deviation_from_median(data):
    """
    find the absolute deviations from median for a list of values
    """
    import numpy as np
    
    mads = []
    med = np.median(data)
    for d in data:
        mads.append(np.abs(d-med))
        
    return mads

In [ ]:
def find_max(data):
    """
    find the index of the maximum value in a list
    """
    from collections import defaultdict

    d = defaultdict(list)
    for i, x in enumerate(data):
        d[x].append(i)

    k = max(d.keys())
    return d[k]


In [ ]:
def plot_pope(files_list, cutoff, num_replicates):
    """
    The function calls a shell script that configures and runs an 
    R script to plot pope plots and extract the top SNPs
    """
    import subprocess
    
    for ID in sorted(files_list):
        print "processing %s:" %ID[0]
        print "data in file: %s" %ID[1]
        c = subprocess.Popen(['sh','plot_pope.sh',ID[1],str(cutoff),ID[0],str(num_replicates)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        (output,err) = c.communicate()

        if err:
            print err
        else:
            print output
        

__Define the function__ that excludes the most extreme replicate.

In [ ]:
def exclude_extreme_rep(dictionary, ids, prefix, cutoff=0):
    """
    Calculate rank statistics while excluding the one most extreme replicate for every factor
    """
    from collections import defaultdict
    import numpy as np
    
    output_columns = ['avg_rank', 'med_rank', 'std_rank', 'var_rank', 'mad_rank', 'avg_rank_rel', 'med_rank_rel', 'var_rank_rel', 'var_rank_weight', 'var_weighted_avg_rank', 'var_weighted_rel_avg_rank']
    glob = defaultdict(dict)
    
    for fac in sorted(ids):
        counts = [dictionary['extremes'][fac][i] for i in sorted(dictionary['extremes'][fac])]
        perc = [float(counts[i])/len(dictionary['global'][fac])*100 for i in range(len(counts))]
        ex = find_max([dictionary['extremes'][fac][i] for i in sorted(dictionary['extremes'][fac])])
        print "\nfactor %s\treplicate %s gave the most extreme ranks for %.2f %% of the SNPs" %(fac, ex[0], perc[ex[0]])

        if perc[ex[0]] > cutoff*100:
            print "will re-calculate stats without replicate %s" %ex[0]
        else:
            print "none of the replicates exceeds the %s threshold" %cutoff
            continue
        
        glob[fac] = defaultdict(dict)
        variances = []
        for SNPid in dictionary['global'][fac].keys():
#            print dictionary['global'][fac][SNPid]['ranks']
            temp_list = dictionary['global'][fac][SNPid]['ranks'][:]
            temp_list.pop(ex[0])
#            print temp_list
            glob[fac][SNPid]['avg_rank'] = np.mean(temp_list)
            glob[fac][SNPid]['med_rank'] = np.median(temp_list)
            glob[fac][SNPid]['std_rank'] = np.std(temp_list)
            glob[fac][SNPid]['var_rank'] = np.var(temp_list)
            glob[fac][SNPid]['mad_rank'] = mad(data=temp_list)
            variances.append(glob[fac][SNPid]['var_rank'])
            glob[fac][SNPid]['avg_rank_rel'] = np.mean(temp_list)/len(dictionary['global'][fac])
            glob[fac][SNPid]['med_rank_rel'] = np.median(temp_list)/len(dictionary['global'][fac])
        #find maximum variance for the current factor
        max_var = max(variances)

        print "Wrting stats to %s" %(prefix+'_'+fac+'_ex_rep_'+str(ex[0])+'.txt')
        OUT = open(prefix+'_'+fac+'_ex_rep_'+str(ex[0])+'.txt','w')
        OUT.write("chrom\tbp\tSNPID\t"+"\t".join(output_columns)+'\n')

        for SNPid in glob[fac].keys(): #calculate relative and weighted variances
            glob[fac][SNPid]['var_rank_rel'] = glob[fac][SNPid]['var_rank'] / max_var
            glob[fac][SNPid]['var_rank_weight'] = 1-glob[fac][SNPid]['var_rank_rel']
            glob[fac][SNPid]['var_weighted_avg_rank'] = glob[fac][SNPid]['var_rank_weight'] * glob[fac][SNPid]['avg_rank']
            glob[fac][SNPid]['var_weighted_rel_avg_rank'] = glob[fac][SNPid]['var_rank_weight'] * glob[fac][SNPid]['avg_rank_rel']

            temp_list = []
#            outstring = str(SNPid)+','
            outstring = dictionary['SNPids'][SNPid]['chrom']+'\t'
            for column in output_columns:
#                print column
#                print glob[fac][SNPid][column]
                temp_list.append(str(glob[fac][SNPid][column]))
            outstring += "\t".join(temp_list)
            OUT.write(outstring+'\n')

        OUT.close()


In [ ]:
def parse_gff(gff):
    """
    parse gff file
    """
    
    gff_dict = {}
    
    gff_fh = open(gff,'r')
    for line in [line.strip() for line in gff_fh]:
#        print line.split('\t')
        if line.split('\t')[2] == 'CDS':
            gene = line.split('\t')[8].split(' ')[3].replace('"','').replace(';','') #This line needs to be adujsted to the gff format
            if not gff_dict.has_key(line.split('\t')[0]):
                gff_dict[line.split('\t')[0]] = {}
            gff_dict[line.split('\t')[0]][line.split('\t')[3]] = gene
            gff_dict[line.split('\t')[0]][line.split('\t')[4]] = gene
            
    return gff_dict    
        

In [ ]:
def find_genes(rank_stats, gff, distance):
    """
    find genes up and downstream of SNPs
    """    
    from collections import defaultdict
    candidates = defaultdict(dict)
    return_dict = defaultdict(dict)
    
    for tsv in rank_stats:
        print "processing rank statistic file: %s" %tsv
        ID = tsv.split('/')[-1].replace('.tsv','')
        candidates[ID] = defaultdict(list)
        rank_stats_fh = open(tsv, 'r')
        rank_stats_fh.readline()
        for SNP in [SNP.strip() for SNP in rank_stats_fh]:
            rank_elem = SNP.split('\t')
            if not candidates[ID].has_key(rank_elem[0]):
                candidates[ID][rank_elem[0]] = []
            candidates[ID][rank_elem[0]].append(rank_elem[1:3])
#            print candidates[tsv][rank_elem[0]]
            

    for tsv in sorted(candidates.keys()):
        print "%s:" %tsv
        for chrom in sorted(candidates[tsv]):
#            print chrom
            for hot in candidates[tsv][chrom]:
                gene_list = []
                temp = []
                nr_genes = []
                lower = int(hot[0])-(distance*1000)
                upper = int(hot[0])+(distance*1000)
#                print "looking at %s" %hot[0]
                if not gff.has_key(chrom):
#                    print "no genes found on %s\n" %chrom
                    continue                
                else:
                    for pos in gff[chrom].keys():
                        temp.append(int(pos))
                    
                for pos in sorted(temp):
                    if pos >= lower and pos <= upper:
#                        print pos,gff[chrom][str(pos)]
                        gene_list.append(gff[chrom][str(pos)])
                    elif pos > upper:
                        break
                 
                nr_genes = list(set(gene_list))
                for unique_gene in nr_genes:
#                    print [chrom,hot[0],hot[1],unique_gene]
                    if not return_dict[tsv].has_key('genes'):
                        return_dict[tsv]['columns'] = ['chrom','bp','ID','gene']
                        return_dict[tsv]['genes'] = []
                    return_dict[tsv]['genes'].append([chrom,hot[0],hot[1],unique_gene])

        if not return_dict.has_key(tsv):
            return_dict[tsv]['genes'] = []
            return_dict[tsv]['columns'] = ['chrom','bp','ID','gene']
        print "identified %i gene(s)" %len(return_dict[tsv]['genes'])
        
    return return_dict

In [ ]:
def annotate_genes(SNPs_to_genes, annotations, whitelist=[]):
    """
    fetch annotation for genes from file produced by Blast2GO
    """
    from collections import defaultdict
    
    annotation = defaultdict(list)
    
    if whitelist:
        for id in whitelist:
            if not SNPs_to_genes.has_key(id):
                raise IOError("You provide an analysis id %s that is not in the dictionary" %id)        
    else:
        whitelist = SNPs_to_genes.keys()[:]

    anno_fh = open(annotations, 'r')
    header = anno_fh.readline().strip().split('\t')
    annotation['header'] = header[1:]
    annotation['genes'] = defaultdict(list)
    for line in [line.strip() for line in anno_fh]:
        annotation['genes'][line.split('\t')[0]] = line.split('\t')[1:]
        
#    for gene in annotation['genes'].keys()[:10]:
#        print gene,annotation['genes'][gene]

    for analysis_id in whitelist:
        print analysis_id
        if len(SNPs_to_genes[analysis_id]['genes']) > 0:
            print "adding annoation for %s" %analysis_id
            for index in range(len(SNPs_to_genes[analysis_id]['genes'])):
                if annotation['genes'].has_key(SNPs_to_genes[analysis_id]['genes'][index][-1]):
                    
                    if len(SNPs_to_genes[analysis_id]['columns']) == 4:
#                        print SNPs_to_genes[analysis_id]['columns']
#                        print "extend the headers"
                        SNPs_to_genes[analysis_id]['columns'].extend(annotation['header'])
#                        print SNPs_to_genes[analysis_id]['columns']
#                    print annotation['genes'][SNPs_to_genes[analysis_id]['genes'][index][-1]]
                    SNPs_to_genes[analysis_id]['genes'][index].extend(annotation['genes'][SNPs_to_genes[analysis_id]['genes'][index][-1]])
                elif len(SNPs_to_genes[analysis_id]['genes'][index]) == 4 and not annotation['genes'].has_key(SNPs_to_genes[analysis_id]['genes'][index][-1]):
                    print "no annoation found for %s" %SNPs_to_genes[analysis_id]['genes'][index][-1]
            
    
        else:
            print "nothing to annotate - 0 candidate genes identified for %s" %analysis_id
    

In [ ]:
def write_candidates(SNPs_to_genes, whitelist=[], rename=[], out_dir='./'):
    """
    write out SNP to candidate genes text files (will be named *.genes.tsv)
    """
    
    if rename:
        if not len(rename) == len(whitelist):
            raise IOError("If you provide a list with new names it needs to be the same length as the whitelist")

    if whitelist:
        for id in whitelist:
            if not SNPs_to_genes.has_key(id):
                raise IOError("You provide an analysis id %s that is not in the dictionary" %id)        
    else:
        whitelist = SNPs_to_genes.keys()[:]
                
    for id in sorted(whitelist):
        print id
        if not len(SNPs_to_genes[id]['genes']) >= 1:
            print "0 candidate genes found"
            continue
        else:
            if len(SNPs_to_genes[id]['columns']) == 4:
                print "writing to: %s" %(out_dir+id+'.genes.tsv')
                out_fh = open(out_dir+id+'.genes.tsv','w')
            else:
                print "writing to: %s" %(out_dir+id+'.genes.annotated.tsv')
                out_fh = open(out_dir+id+'.genes.annotated.tsv','w')

            out_fh.write("%s\n" %"\t".join(SNPs_to_genes[id]['columns']))
            for gene in SNPs_to_genes[id]['genes']:
                out_fh.write("%s\n" %"\t".join(gene))
            
        out_fh.close()